In [1]:
# !pip install clouddrift numpy s3fs xarray

In [2]:
import clouddrift as cd
import numpy as np
import s3fs
import xarray as xr

## Access EDITO s3

In [3]:
s3 = s3fs.S3FileSystem(
  anon=True,
  endpoint_url="https://minio.lab.dive.edito.eu"
)

In [4]:
s3.ls("/oidc-bertrava/data")

['oidc-bertrava/data/4dvarnet.zarr',
 'oidc-bertrava/data/duacs.zarr',
 'oidc-bertrava/data/gdp6h.zarr',
 'oidc-bertrava/data/miost.zarr',
 'oidc-bertrava/data/vardyn.zarr']

## Sea surface currents

In [5]:
vardyn_store = s3fs.S3Map(root=f"s3://oidc-bertrava/data/vardyn.zarr", s3=s3)

In [6]:
ds_vardyn = xr.open_zarr(vardyn_store)
ds_vardyn

<xarray.Dataset> Size: 4GB
Dimensions:             (time: 465, latitude: 276, longitude: 600)
Coordinates:
  * latitude            (latitude) float32 1kB 25.0 25.09 25.18 ... 49.91 50.0
  * longitude           (longitude) float64 5kB 280.0 280.1 ... 349.8 349.9
  * time                (time) datetime64[ns] 4kB 2023-07-27 ... 2023-11-20
Data variables:
    adt                 (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    relative_vorticity  (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    sla                 (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    ugos                (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    ugosa               (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    vgos                (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    vgosa               (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>

Longitudes range from 0° to 360°: need to transform them to work in [-180°, 180°].

In [7]:
ds_vardyn["longitude"] = ds_vardyn.longitude - 180
ds_vardyn

<xarray.Dataset> Size: 4GB
Dimensions:             (time: 465, latitude: 276, longitude: 600)
Coordinates:
  * latitude            (latitude) float32 1kB 25.0 25.09 25.18 ... 49.91 50.0
  * longitude           (longitude) float64 5kB 100.0 100.1 ... 169.8 169.9
  * time                (time) datetime64[ns] 4kB 2023-07-27 ... 2023-11-20
Data variables:
    adt                 (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    relative_vorticity  (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    sla                 (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    ugos                (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    ugosa               (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    vgos                (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    vgosa               (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>

### GDP Drifters

MIOST, VarDyn, 4dVarNet datasets from Aviso+ start at 2023-07-27: we will have to use the GDP6H dataset.

In [8]:
gdp6h_store = s3fs.S3Map(root=f"s3://oidc-bertrava/data/gdp6h.zarr", s3=s3)

In [9]:
ds_gdp6h = xr.open_zarr(gdp6h_store)
ds_gdp6h

<xarray.Dataset> Size: 2GB
Dimensions:                (traj: 27647, obs: 46535470)
Coordinates:
    id                     (traj) int64 221kB dask.array<chunksize=(27647,), meta=np.ndarray>
    time                   (obs) datetime64[ns] 372MB dask.array<chunksize=(181780,), meta=np.ndarray>
Dimensions without coordinates: traj, obs
Data variables: (12/49)
    BuoyTypeManufacturer   (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    BuoyTypeSensorArray    (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    CurrentProgram         (traj) float64 221kB dask.array<chunksize=(27647,), meta=np.ndarray>
    DeployingCountry       (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    DeployingShip          (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    DeploymentComments     (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    ...                     ...
    start_lon              (traj) float32 111kB dask.array<chunksize=(27647,), meta=np.ndarray>
    temp                   (obs) float32 186MB dask.array<chunksize=(363559,), meta=np.ndarray>
    typebuoy               (traj) |S10 276kB dask.array<chunksize=(13824,), meta=np.ndarray>
    typedeath              (traj) int8 28kB dask.array<chunksize=(27647,), meta=np.ndarray>
    ve                     (obs) float32 186MB dask.array<chunksize=(363559,), meta=np.ndarray>
    vn                     (obs) float32 186MB dask.array<chunksize=(363559,), meta=np.ndarray>
Attributes: (12/18)
    Conventions:          CF-1.6
    acknowledgement:      Lumpkin, Rick; Centurioni, Luca (2019). NOAA Global...
    contributor_name:     NOAA Global Drifter Program
    contributor_role:     Data Acquisition Center
    date_created:         2024-04-04T13:44:01.176967
    doi:                  10.25921/7ntx-z961
    ...                   ...
    publisher_name:       GDP Drifter DAC
    publisher_url:        https://www.aoml.noaa.gov/phod/gdp
    summary:              Global Drifter Program six-hourly data
    time_coverage_end:    2023-10-18:18:00:00Z
    time_coverage_start:  1979-02-15:00:00:00Z
    title:                Global Drifter Program drifting buoy collection

In [10]:
ds_drifters = cd.ragged.subset(
    ds_gdp6h, 
    {
        "time": lambda t: (t >= np.datetime64("2023-07-27")) & (t <= np.datetime64("2023-11-20")),
        "lat": lambda l: (l >= 25) & (l <= 50),
        "lon": lambda l: (l >= 100) & (l <= 169.9)
    }, 
    row_dim_name="traj"
)
ds_drifters

<xarray.Dataset> Size: 582kB
Dimensions:                (traj: 50, obs: 11502)
Coordinates:
    id                     (traj) int64 400B dask.array<chunksize=(50,), meta=np.ndarray>
    time                   (obs) datetime64[ns] 92kB dask.array<chunksize=(11502,), meta=np.ndarray>
Dimensions without coordinates: traj, obs
Data variables: (12/49)
    BuoyTypeManufacturer   (traj) |S20 1kB dask.array<chunksize=(50,), meta=np.ndarray>
    BuoyTypeSensorArray    (traj) |S20 1kB dask.array<chunksize=(50,), meta=np.ndarray>
    CurrentProgram         (traj) float64 400B dask.array<chunksize=(50,), meta=np.ndarray>
    DeployingCountry       (traj) |S20 1kB dask.array<chunksize=(50,), meta=np.ndarray>
    DeployingShip          (traj) |S20 1kB dask.array<chunksize=(50,), meta=np.ndarray>
    DeploymentComments     (traj) |S20 1kB dask.array<chunksize=(50,), meta=np.ndarray>
    ...                     ...
    start_lon              (traj) float32 200B dask.array<chunksize=(50,), meta=np.ndarray>
    temp                   (obs) float32 46kB dask.array<chunksize=(11502,), meta=np.ndarray>
    typebuoy               (traj) |S10 500B dask.array<chunksize=(50,), meta=np.ndarray>
    typedeath              (traj) int8 50B dask.array<chunksize=(50,), meta=np.ndarray>
    ve                     (obs) float32 46kB dask.array<chunksize=(11502,), meta=np.ndarray>
    vn                     (obs) float32 46kB dask.array<chunksize=(11502,), meta=np.ndarray>
Attributes: (12/18)
    Conventions:          CF-1.6
    acknowledgement:      Lumpkin, Rick; Centurioni, Luca (2019). NOAA Global...
    contributor_name:     NOAA Global Drifter Program
    contributor_role:     Data Acquisition Center
    date_created:         2024-04-04T13:44:01.176967
    doi:                  10.25921/7ntx-z961
    ...                   ...
    publisher_name:       GDP Drifter DAC
    publisher_url:        https://www.aoml.noaa.gov/phod/gdp
    summary:              Global Drifter Program six-hourly data
    time_coverage_end:    2023-10-18:18:00:00Z
    time_coverage_start:  1979-02-15:00:00:00Z
    title:                Global Drifter Program drifting buoy collection